# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from random import sample

import re

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

### Note: This Step is for the extraction of California State and Foods category 

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'FOODS')]

In [5]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1612,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,3,0,0,1,...,0,2,0,4,1,1,0,1,1,0
1613,FOODS_1_002_CA_1_validation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,0,1,0,1,...,1,3,1,0,0,1,2,0,0,0
1614,FOODS_1_003_CA_1_validation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,3,0,2,1,1,0,1,0,1,0
1615,FOODS_1_004_CA_1_validation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1616,FOODS_1_005_CA_1_validation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,3,9,3,3,...,3,1,1,2,0,2,2,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12191,FOODS_3_823_CA_4_validation,FOODS_3_823,FOODS_3,FOODS,CA_4,CA,1,1,1,4,...,3,1,2,2,0,1,1,3,3,0
12192,FOODS_3_824_CA_4_validation,FOODS_3_824,FOODS_3,FOODS,CA_4,CA,0,0,0,1,...,0,1,1,1,0,2,0,0,0,1
12193,FOODS_3_825_CA_4_validation,FOODS_3_825,FOODS_3,FOODS,CA_4,CA,2,3,2,1,...,0,2,0,1,3,1,1,0,2,0
12194,FOODS_3_826_CA_4_validation,FOODS_3_826,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,1,2,2,1,4,5,1,7,0,4


In [6]:
INPUT_DIR_2 = r'/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/List of Product ID according to 4 demand patterns/California/Foods'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [7]:
len(sales_intermittent)

1301

In [8]:
len(sales_lumpy)

395

In [9]:
len(sales_erratic)

458

In [10]:
len(sales_smooth)

3594

### Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)

In [11]:
sales_smooth

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1612,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,3,0,0,1,...,0,2,0,4,1,1,0,1,1,0
1613,FOODS_1_002_CA_1_validation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,0,1,0,1,...,1,3,1,0,0,1,2,0,0,0
1614,FOODS_1_003_CA_1_validation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,3,0,2,1,1,0,1,0,1,0
1615,FOODS_1_004_CA_1_validation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1617,FOODS_1_006_CA_1_validation,FOODS_1_006,FOODS_1,FOODS,CA_1,CA,0,0,3,1,...,1,3,1,0,2,1,2,3,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12186,FOODS_3_818_CA_4_validation,FOODS_3_818,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,4,2,1,7,5,2,6,5,5
12187,FOODS_3_819_CA_4_validation,FOODS_3_819,FOODS_3,FOODS,CA_4,CA,4,4,3,1,...,0,2,0,1,1,1,1,0,0,0
12188,FOODS_3_820_CA_4_validation,FOODS_3_820,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,1,1,2,0,4,2,5,1,3
12190,FOODS_3_822_CA_4_validation,FOODS_3_822,FOODS_3,FOODS,CA_4,CA,1,3,2,0,...,2,0,0,1,0,1,1,0,1,2


# User-defined functions to calculate Metrics and Croston algorithm

In [12]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [13]:
def Croston(ts,extra_periods=1,alpha=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
    # Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = alpha*q + (1-alpha)*p[t]
            f[t+1] = a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Format sales data and then merge with calendar data

In [14]:
sales_pattern = sales_smooth.copy()

In [15]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

# Set value for parameters

In [16]:
list_params = [round(item, 2) for item in list(np.arange(0.05, 0.95, 0.05))]

In [17]:
list_params

[0.05,
 0.1,
 0.15,
 0.2,
 0.25,
 0.3,
 0.35,
 0.4,
 0.45,
 0.5,
 0.55,
 0.6,
 0.65,
 0.7,
 0.75,
 0.8,
 0.85,
 0.9]

In [18]:
start_train_week = 11318
end_train_week = 11610
n_pred_weeks = 2
num_train_needed = 50
validation_training_ratio = 0.95
num_id_sample_for_tuning = 200

In [19]:
sample_list = sample(list_pattern_py, num_id_sample_for_tuning)

In [20]:
dataframe_check = dataframe_weekly[sample_list].iloc[-2:].T

dataframe_check['Check'] = np.where((dataframe_check[271] == 0) &
                                    (dataframe_check[272] == 0), 1, 0)

sample_removed = dataframe_check[dataframe_check['Check'] == 1].index.tolist()

In [21]:
sample_list_filtered = [x for x in sample_list if (x not in sample_removed)]

In [22]:
len(sample_list_filtered)

151

# Hyperparameter tuning for Croston

In [23]:
train_data = dataframe_weekly[(dataframe_weekly['wm_yr_wk'] >= start_train_week) & (dataframe_weekly['wm_yr_wk'] <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]

In [24]:
start = timer()

df_result_all_params = pd.DataFrame()
for product in sample_list_filtered:
    print('Currently Running: %s' % product)
    print('Progressing: {0} %'.format(round(sample_list_filtered.index(product) / len(sample_list_filtered) * 100, 2)))

    train_first = train_data_first_part[product]
    train_second = train_data_second_part[product]
    for i in list_params:
        predictions = list()
        history = [x for x in train_first[num_train_needed*-1:]]
        for t in train_second.index.tolist():
            yhat = Croston(history, extra_periods=1, alpha=i)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(train_second[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                        'Actual Data': train_second,
                                        'Forecast': predictions,
                                        'Alpha': [i for count in range(len(train_second))]})
        df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: FOODS_2_217_CA_3
Progressing: 0.0 %
Currently Running: FOODS_3_544_CA_1
Progressing: 0.66 %
Currently Running: FOODS_2_153_CA_4
Progressing: 1.32 %
Currently Running: FOODS_1_081_CA_3
Progressing: 1.99 %
Currently Running: FOODS_3_162_CA_2
Progressing: 2.65 %
Currently Running: FOODS_3_232_CA_2
Progressing: 3.31 %
Currently Running: FOODS_2_047_CA_1
Progressing: 3.97 %
Currently Running: FOODS_3_620_CA_3
Progressing: 4.64 %
Currently Running: FOODS_3_320_CA_3
Progressing: 5.3 %
Currently Running: FOODS_3_428_CA_3
Progressing: 5.96 %
Currently Running: FOODS_1_024_CA_2
Progressing: 6.62 %
Currently Running: FOODS_3_547_CA_1
Progressing: 7.28 %
Currently Running: FOODS_3_677_CA_2
Progressing: 7.95 %
Currently Running: FOODS_3_765_CA_4
Progressing: 8.61 %
Currently Running: FOODS_3_310_CA_2
Progressing: 9.27 %
Currently Running: FOODS_2_179_CA_1
Progressing: 9.93 %
Currently Running: FOODS_1_208_CA_3
Progressing: 10.6 %
Currently Running: FOODS_3_023_CA_1
Progressing: 1

In [25]:
df_result_all_params

,Product,Actual Data,Forecast,Alpha
0,FOODS_2_217_CA_3,9.0,7.389885,0.05
1,FOODS_2_217_CA_3,9.0,7.464466,0.05
2,FOODS_2_217_CA_3,8.0,7.535855,0.05
3,FOODS_2_217_CA_3,14.0,7.557510,0.05
4,FOODS_2_217_CA_3,7.0,7.859093,0.05
...,...,...,...,...
21739,FOODS_3_476_CA_2,42.0,47.291867,0.90
21740,FOODS_3_476_CA_2,33.0,42.529187,0.90
21741,FOODS_3_476_CA_2,28.0,33.952919,0.90
21742,FOODS_3_476_CA_2,23.0,28.595292,0.90


# Summay all metrics

In [26]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [27]:
start = timer()

df_result_metrics_all_params = df_result_all_params.groupby(['Product', 'Alpha']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.09202240211666653 minutes


In [28]:
df_result_metrics_all_params

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_024_CA_2,0.05,1.305061,34.0914,34.0519,37.2874,4.474496,5.023024
1,FOODS_1_024_CA_2,0.10,1.410638,36.8493,36.5146,42.0350,4.836472,5.293805
2,FOODS_1_024_CA_2,0.15,1.377734,35.9898,35.7090,40.0374,4.723659,5.317199
3,FOODS_1_024_CA_2,0.20,1.294135,33.8060,33.6196,36.3568,4.437033,5.246065
4,FOODS_1_024_CA_2,0.25,1.191005,31.1120,30.9369,32.3579,4.083446,5.143766
...,...,...,...,...,...,...,...,...
2713,FOODS_3_817_CA_3,0.70,0.803037,58.2703,58.3504,97.6917,4.588784,5.436381
2714,FOODS_3_817_CA_3,0.75,0.828668,60.1301,59.9901,99.7260,4.735248,5.551654
2715,FOODS_3_817_CA_3,0.80,0.857138,62.1960,61.8048,101.8924,4.897934,5.674464
2716,FOODS_3_817_CA_3,0.85,0.886814,64.3493,63.7017,104.1185,5.067509,5.806402


# Check what products has unexpected metrics result

In [29]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
1476,FOODS_3_323_CA_1,0.05,inf,inf,200.0,NaN,3.651990,3.651990
1477,FOODS_3_323_CA_1,0.10,inf,inf,200.0,NaN,4.850448,4.850448
1478,FOODS_3_323_CA_1,0.15,inf,inf,200.0,NaN,4.905288,4.905288
1479,FOODS_3_323_CA_1,0.20,inf,inf,200.0,NaN,4.691712,4.691712
1480,FOODS_3_323_CA_1,0.25,inf,inf,200.0,NaN,4.491145,4.491145
1481,FOODS_3_323_CA_1,0.30,inf,inf,200.0,NaN,4.346227,4.346227
1482,FOODS_3_323_CA_1,0.35,inf,inf,200.0,NaN,4.249469,4.249469
1483,FOODS_3_323_CA_1,0.40,inf,inf,200.0,NaN,4.184191,4.184191
1484,FOODS_3_323_CA_1,0.45,inf,inf,200.0,NaN,4.134452,4.134452
1485,FOODS_3_323_CA_1,0.50,inf,inf,200.0,NaN,4.087679,4.087679


In [30]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
1476,FOODS_3_323_CA_1,0.05,inf,inf,200.0,NaN,3.651990,3.651990
1477,FOODS_3_323_CA_1,0.10,inf,inf,200.0,NaN,4.850448,4.850448
1478,FOODS_3_323_CA_1,0.15,inf,inf,200.0,NaN,4.905288,4.905288
1479,FOODS_3_323_CA_1,0.20,inf,inf,200.0,NaN,4.691712,4.691712
1480,FOODS_3_323_CA_1,0.25,inf,inf,200.0,NaN,4.491145,4.491145
1481,FOODS_3_323_CA_1,0.30,inf,inf,200.0,NaN,4.346227,4.346227
1482,FOODS_3_323_CA_1,0.35,inf,inf,200.0,NaN,4.249469,4.249469
1483,FOODS_3_323_CA_1,0.40,inf,inf,200.0,NaN,4.184191,4.184191
1484,FOODS_3_323_CA_1,0.45,inf,inf,200.0,NaN,4.134452,4.134452
1485,FOODS_3_323_CA_1,0.50,inf,inf,200.0,NaN,4.087679,4.087679


In [31]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
1458,FOODS_3_321_CA_4,0.05,1.089912,99.6491,NaN,99.7807,0.622807,1.451721
1459,FOODS_3_321_CA_4,0.10,1.089912,99.6491,NaN,99.7807,0.622807,1.451721
1460,FOODS_3_321_CA_4,0.15,1.089912,99.6491,NaN,99.7807,0.622807,1.451721
1461,FOODS_3_321_CA_4,0.20,1.089912,99.6491,NaN,99.7807,0.622807,1.451721
1462,FOODS_3_321_CA_4,0.25,1.089912,99.6491,NaN,99.7807,0.622807,1.451721
...,...,...,...,...,...,...,...,...
2659,FOODS_3_795_CA_1,0.70,1.773285,93.1146,NaN,93.3258,4.306550,6.339172
2660,FOODS_3_795_CA_1,0.75,1.730541,90.8701,NaN,91.1028,4.202743,6.217605
2661,FOODS_3_795_CA_1,0.80,1.665396,87.4494,NaN,87.7092,4.044532,6.050311
2662,FOODS_3_795_CA_1,0.85,1.563863,82.1179,NaN,82.4242,3.797954,5.833985


In [32]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
1476,FOODS_3_323_CA_1,0.05,inf,inf,200.0,NaN,3.651990,3.651990
1477,FOODS_3_323_CA_1,0.10,inf,inf,200.0,NaN,4.850448,4.850448
1478,FOODS_3_323_CA_1,0.15,inf,inf,200.0,NaN,4.905288,4.905288
1479,FOODS_3_323_CA_1,0.20,inf,inf,200.0,NaN,4.691712,4.691712
1480,FOODS_3_323_CA_1,0.25,inf,inf,200.0,NaN,4.491145,4.491145
1481,FOODS_3_323_CA_1,0.30,inf,inf,200.0,NaN,4.346227,4.346227
1482,FOODS_3_323_CA_1,0.35,inf,inf,200.0,NaN,4.249469,4.249469
1483,FOODS_3_323_CA_1,0.40,inf,inf,200.0,NaN,4.184191,4.184191
1484,FOODS_3_323_CA_1,0.45,inf,inf,200.0,NaN,4.134452,4.134452
1485,FOODS_3_323_CA_1,0.50,inf,inf,200.0,NaN,4.087679,4.087679


In [33]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [34]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [35]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [36]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 0.6622516556291391 %
Percentage of unexpected values of WMAPE is: 0.6622516556291391 %
Percentage of unexpected values of SMAPE is: 4.635761589403973 %
Percentage of unexpected values of MAPE is: 0.6622516556291391 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [37]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [38]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.05,1.256305,57.709722,63.039274,70.125159,9.510676,10.960397
0.10,1.129441,52.387427,56.255015,63.038028,8.496001,10.017536
0.15,1.057540,49.546941,52.665181,59.906713,7.984833,9.454025
0.20,1.013666,47.858013,50.580107,58.255188,7.651170,9.138268
0.25,0.982404,46.695092,49.343187,57.218736,7.395177,8.919746
0.30,0.960360,45.827371,48.633425,56.627233,7.189286,8.757608
0.35,0.945052,45.190962,48.230569,56.341229,7.045003,8.638837
0.40,0.936020,44.812014,48.040630,56.276885,6.978697,8.557259
0.45,0.930860,44.631764,48.021270,56.366111,6.941196,8.507319


In [39]:
for mts in list_metrics:
    print('The optimum Alpha based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha based on Mean MASE is: 0.5
The optimum Alpha based on Mean WMAPE is: 0.5
The optimum Alpha based on Mean SMAPE is: 0.45
The optimum Alpha based on Mean MAPE is: 0.4
The optimum Alpha based on Mean MAE is: 0.5
The optimum Alpha based on Mean RMSE is: 0.55


### Get MEDIAN metrics value of each Alpha

In [40]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.05,0.906787,43.0828,47.6743,49.3558,4.687654,5.579814
0.10,0.880460,40.6037,42.6040,43.2506,4.297219,5.203490
0.15,0.872383,39.8160,40.4233,40.3886,3.986825,5.132260
0.20,0.827515,39.0943,40.1356,39.2376,4.097552,5.061485
0.25,0.836583,38.6023,38.6274,39.6382,4.010254,4.964349
0.30,0.845318,37.7333,38.1275,40.1123,3.921324,4.948685
0.35,0.843389,36.7937,37.2052,39.4019,3.950810,4.915329
0.40,0.848719,36.7476,37.1371,39.8323,3.955356,4.880661
0.45,0.846691,36.4170,36.8654,40.1828,3.962925,4.930538


In [41]:
for mts in list_metrics:
    print('The optimum Alpha based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha based on Median of MASE is: 0.2
The optimum Alpha based on Median of WMAPE is: 0.5
The optimum Alpha based on Median of SMAPE is: 0.45
The optimum Alpha based on Median of MAPE is: 0.2
The optimum Alpha based on Median of MAE is: 0.3
The optimum Alpha based on Median of RMSE is: 0.4


# Run Croston model after decide best paramaters

In [42]:
best_alpha = 0.4

In [43]:
dataframe_weekly

,wm_yr_wk,FOODS_1_001_CA_1,FOODS_1_001_CA_2,FOODS_1_001_CA_3,FOODS_1_001_CA_4,FOODS_1_002_CA_1,FOODS_1_002_CA_2,FOODS_1_002_CA_3,FOODS_1_002_CA_4,FOODS_1_003_CA_1,...,FOODS_3_821_CA_1,FOODS_3_821_CA_3,FOODS_3_822_CA_1,FOODS_3_822_CA_4,FOODS_3_825_CA_1,FOODS_3_826_CA_1,FOODS_3_826_CA_2,FOODS_3_826_CA_4,FOODS_3_827_CA_1,FOODS_3_827_CA_2
0,11101,5.0,19.0,8.0,0.0,1.0,6.0,4.0,3.0,0.0,...,3.0,3.0,31.0,15.0,6.0,0.0,2.0,9.0,0.0,0.0
1,11102,1.0,3.0,12.0,5.0,2.0,4.0,2.0,3.0,0.0,...,5.0,2.0,24.0,10.0,5.0,0.0,6.0,9.0,0.0,0.0
2,11103,0.0,1.0,6.0,2.0,0.0,5.0,0.0,4.0,2.0,...,6.0,7.0,38.0,13.0,9.0,12.0,7.0,18.0,0.0,0.0
3,11104,1.0,8.0,0.0,1.0,0.0,2.0,3.0,5.0,5.0,...,4.0,1.0,25.0,14.0,7.0,0.0,11.0,10.0,0.0,0.0
4,11105,0.0,0.0,3.0,0.0,0.0,7.0,2.0,1.0,3.0,...,7.0,4.0,27.0,14.0,8.0,8.0,7.0,12.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,11608,6.0,13.0,6.0,2.0,0.0,6.0,3.0,0.0,6.0,...,0.0,7.0,32.0,18.0,2.0,9.0,4.0,17.0,0.0,0.0
269,11609,1.0,9.0,1.0,3.0,0.0,6.0,0.0,1.0,6.0,...,0.0,4.0,54.0,12.0,8.0,15.0,10.0,9.0,0.0,0.0
270,11610,10.0,9.0,11.0,1.0,1.0,4.0,2.0,5.0,16.0,...,0.0,2.0,53.0,20.0,5.0,13.0,11.0,12.0,0.0,0.0
271,11611,8.0,7.0,1.0,7.0,3.0,3.0,1.0,1.0,8.0,...,4.0,10.0,54.0,18.0,12.0,23.0,1.0,13.0,0.0,0.0


In [44]:
train_data = dataframe_weekly[(dataframe_weekly.wm_yr_wk >= start_train_week) & (dataframe_weekly.wm_yr_wk <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

In [45]:
df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    train_product = train_data[product]
    test_product = test_data[product]
    predictions = list()
    history = [x for x in train_product[num_train_needed*-1:]]
    for t in test_product.index.tolist():
        yhat = Croston(history, extra_periods=1, alpha=best_alpha)['Forecast'].iloc[-1]
        predictions.append(yhat)
        history.append(test_product[t])
    df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                    'Actual Data': test_product,
                                    'Forecast': predictions})
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)

In [46]:
df_result_best_params

,Product,Actual Data,Forecast
0,FOODS_1_001_CA_1,8.0,6.017378
1,FOODS_1_001_CA_1,3.0,6.810427
2,FOODS_1_002_CA_1,3.0,0.492829
3,FOODS_1_002_CA_1,0.0,1.033997
4,FOODS_1_003_CA_1,8.0,10.319291
...,...,...,...
7183,FOODS_3_820_CA_4,5.0,3.031405
7184,FOODS_3_822_CA_4,18.0,16.886828
7185,FOODS_3_822_CA_4,18.0,17.332097
7186,FOODS_3_826_CA_4,13.0,11.533094


In [47]:
df_result_final = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [48]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_001_CA_1,0.579305,52.6641,52.9846,75.8985,2.896524,3.037280
1,FOODS_1_001_CA_2,0.679957,23.9985,23.5780,26.1830,2.039871,2.143827
2,FOODS_1_001_CA_3,0.653031,106.8595,109.6654,368.6595,5.877275,5.963274
3,FOODS_1_001_CA_4,0.633540,126.7080,150.1307,66.7828,4.434781,4.441271
4,FOODS_1_002_CA_1,0.590195,118.0389,171.7805,83.5724,1.770584,1.917688
...,...,...,...,...,...,...,...
3589,FOODS_3_826_CA_1,0.715752,34.8204,36.9363,32.4443,6.441769,7.222926
3590,FOODS_3_826_CA_2,0.869084,124.1549,82.1795,428.8264,4.345422,6.049131
3591,FOODS_3_826_CA_4,0.695588,26.0846,28.0879,23.7476,4.173529,4.974349
3592,FOODS_3_827_CA_1,NaN,NaN,NaN,NaN,0.000000,0.000000


In [49]:
df_result_final.isnull().sum()

Product      0
MASE       711
WMAPE      711
SMAPE      712
MAPE       829
MAE          0
RMSE         0
dtype: int64

In [50]:
df_result_final.MAPE.mean()

61.289599493670885

In [51]:
df_result_final.to_csv('Croston_Smooth_Metrics.csv')